# Rethinking Network Design and Local Geometry in Point Cloud: A Simple Residual MLP Framework

## Paper Reviews

### Prior Reseach

**PointNet++** 는 Point Cloud을 접근하기 위한 base pipeline로서 매우 성공적인 결과를 얻었음  
이를 이용하여 local set의 geometrical feature를 얻기 위해 Convolution, Self-Attention, GCN 등 다양한 방법이 제시됨  
이는 필연적으로 complex, high-computational한 structure를 유도할 수 밖에 없는데 상대적으로 performance gain이 매우 적었음  
→ practical한 application에서는 PointNet++를 제외하고 거의 사용될 수 없는 문제가 발생함

∴ local set representation을 개선하는 것보다 다른 방향에서의 performance gain이 필요함  
→ 상대적으로 가벼운 MLP layer로 구성하여 layer stack을 가능하게 하는 것이 empirical하게 중요하다는 것을 **PointMLP**로 보임

<p align="center">
<img width="724" alt="1" src="https://user-images.githubusercontent.com/86907286/195959300-8ab329c6-8c5b-4915-b95c-a862e643e0a8.png">
</p>


### Revisiting PointNet++ Architecture

**PointNet++** 의 영향 하에 있는 Architecture들은 모두 하나의 local subset에 대해 다음과 같은 operation이 stacking된 형태라고 볼 수 있음  
이때 $j = 1, \cdots, K$ sample point를 갖는 $i$ group에서의 aggregation function $\mathcal{A}(\cdot)$은 PointNet++의 경우 max-pooling으로 구현되었음  

$$ g_i = \mathcal{A}(\Phi(f_{i, j}|j = 1, \cdots, K) $$

한편 $\Phi(\cdot)$의 경우 실질적인 local feature extraction을 담당하는 function이며, 이것은 자유롭게 plug-and-play가 가능했음  
따라서 기존의 연구는 이 부분을 다른 영역에서 성공적이었던 feature extraction method를 적용했지만 이로 인해 발생한 gain은 크다고 하지 못했음  
이렇게 plug-and-play로 적용된 $\Phi(\cdot)$ 들은 기본적으로 local feature를 복잡한 방식으로 extraction하는 방법들임  
하지만 이러한 방법들이 성공적이지 못했다는 것은 **Point Cloud에서 local geometric이 그렇게 중요하지 않은 정보**라는 것을 암시함  

∴ 그만큼의 computation cost을 **Netwrok를 deep하게 가져가는 것으로 전환하여** 다른 방향으로의 performance gain을 얻어야 할 필요가 있음

### Deep Residual MLP

이를 위해 **적극적인 Residual Connection과 MLP의 stacking을 적용할 수 있는 구조**를 반영하는 **PointMLP**를 구축    
MLP는 PointNet이 그랬던 것처럼 permutation invariant하면서 동시에 낮은 cost로 residual connection을 통해 쉽게 stacking이 가능함  
이를 통해 aggregation이 발생할 때 다음과 같이 local representation 외에 **aggregation representation을 한번 더 check**하는 것을 실현시킬 수 있음 

$$ g_i = \Phi_{\text{post}}(\mathcal{A}(\Phi_{\text{pre}}(f_{i, j}|j = 1, \cdots, K)) $$

이때 pre-block $\Phi_{\text{pre}}(\cdot)$는 3개 layer, post-block $\Phi_{\text{post}}(\cdot)$는 2개 layer의 MLP로 구성되어 총 4번의 block stack으로 구현됨  
따라서 depth를 조절하면서 3개의 classification layer 및 1개의 coordinate의 embedding을 위한 합쳐 $L$ layer를 갖는 deep한 architecture를 학습  

$$ L = 1 + \sum^L_{i=1} (1 + 3 \times |\Phi_{\text{pre}}(\cdot)| + 2 \times |\Phi_{\text{post}}(\cdot)|) $$

<p align="center">
<img width="484" alt="2" src="https://user-images.githubusercontent.com/86907286/195959302-5a44528a-a9fa-4f76-a570-69257fb25560.png">
</p>

또한 stacking으로 구현된다는 점에서 computational cost를 절약하기 위해서 기존의 MLP에서 사용되던 **bottleneck**을 쉽게 적용할 수 있음   
empirical하게 bottleneck을 도입해도 performance 하락이 사실상 없었으며 낮은 computational cost와 동시에 좋은 performance를 확보할 수 있었음

<p align="center">
<img width="740" alt="3" src="https://user-images.githubusercontent.com/86907286/195959305-f4e1bb53-1e63-4946-9a97-947db720d36e.png">
</p>

### Geometric Affine Layer

deep stacking을 위해 상대적으로 local geometry에 대해서 덜 집중하기 위해서 MLP를 도입했지만 이를 완전히 무시할 수는 없음  
특히 Point Cloud라는 점에서 deep residual network가 모든 point에 대해서 공유되므로 sparse, irregular geometry에 대해서 robust하지 못함  

∴ 이러한 문제를 보상하기 위해 local geometry에 대한 affine transform으로 normalizing하는 것을 통해 stability를 확보  
→ 학습되는 parameter $\alpha$, $\beta$를 통해 batch normalization과 같이 geometry에 대한 smoothing 효과를 유도

$$ \{ \hat{f}_{i, j} \} = \alpha \odot \frac{\{ f_{i, j} - f_i \}}{\sigma + \epsilon} + \beta, \,\,\,\, \sigma = \sqrt{\frac{1}{k \times n \times d} \sum^n_{i=1} \sum^k_{j=1} (f_{i, j} - f_i)^2} $$ 

<p align="center">
<img width="737" alt="4" src="https://user-images.githubusercontent.com/86907286/195959307-faa96dc3-1293-4f1e-b588-49fb1bd120a4.png">
</p>

## Implementation Reviews

pre/post-block with Residual Connection

In [ ]:
class ConvBNReLURes1D(nn.Module):
    def __init__(self, channel, kernel_size=1, groups=1, res_expansion=1.0, bias=True, activation='relu'):
        super(ConvBNReLURes1D, self).__init__()
        self.act = get_activation(activation)
        self.net1 = nn.Sequential(
            nn.Conv1d(in_channels=channel, out_channels=int(channel * res_expansion),
                      kernel_size=kernel_size, groups=groups, bias=bias),
            nn.BatchNorm1d(int(channel * res_expansion)),
            self.act
        )
        if groups > 1:
            self.net2 = nn.Sequential(
                nn.Conv1d(in_channels=int(channel * res_expansion), out_channels=channel,
                          kernel_size=kernel_size, groups=groups, bias=bias),
                nn.BatchNorm1d(channel),
                self.act,
                nn.Conv1d(in_channels=channel, out_channels=channel,
                          kernel_size=kernel_size, bias=bias),
                nn.BatchNorm1d(channel),
            )
        else:
            self.net2 = nn.Sequential(
                nn.Conv1d(in_channels=int(channel * res_expansion), out_channels=channel,
                          kernel_size=kernel_size, bias=bias),
                nn.BatchNorm1d(channel)
            )

    def forward(self, x):
        return self.act(self.net2(self.net1(x)) + x)

class PreExtraction(nn.Module):
    def __init__(self, channels, out_channels,  blocks=1, groups=1, res_expansion=1, bias=True,
                 activation='relu', use_xyz=True):
        """
        input: [b,g,k,d]: output:[b,d,g]
        :param channels:
        :param blocks:
        """
        super(PreExtraction, self).__init__()
        in_channels = 3+2*channels if use_xyz else 2*channels
        self.transfer = ConvBNReLU1D(in_channels, out_channels, bias=bias, activation=activation)
        operation = []
        for _ in range(blocks):
            operation.append(
                ConvBNReLURes1D(out_channels, groups=groups, res_expansion=res_expansion,
                                bias=bias, activation=activation)
            )
        self.operation = nn.Sequential(*operation)

    def forward(self, x):
        b, n, s, d = x.size()  # torch.Size([32, 512, 32, 6])
        x = x.permute(0, 1, 3, 2)
        x = x.reshape(-1, d, s)
        x = self.transfer(x)
        batch_size, _, _ = x.size()
        x = self.operation(x)  # [b, d, k]
        x = F.adaptive_max_pool1d(x, 1).view(batch_size, -1)
        x = x.reshape(b, n, -1).permute(0, 2, 1)
        return x


class PosExtraction(nn.Module):
    def __init__(self, channels, blocks=1, groups=1, res_expansion=1, bias=True, activation='relu'):
        """
        input[b,d,g]; output[b,d,g]
        :param channels:
        :param blocks:
        """
        super(PosExtraction, self).__init__()
        operation = []
        for _ in range(blocks):
            operation.append(
                ConvBNReLURes1D(channels, groups=groups, res_expansion=res_expansion, bias=bias, activation=activation)
            )
        self.operation = nn.Sequential(*operation)

    def forward(self, x):  # [b, d, g]
        return self.operation(x))

## Reference

https://arxiv.org/abs/2202.07123  
https://github.com/ma-xu/pointMLP-pytorch/blob/b5dcf4d0ca2085d10fe1dc5e6f2972cddec8399d/classification_ModelNet40/models/pointmlp.py 